# Chapter 10 - Artificial Neural Networks

## Perceptrons

### Using Scikit-Learn

In [ ]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.linear_model import Perceptron

iris = load_iris()
X = iris.data[:, (2, 3)]
y = (iris.target == 0).astype(np.int)

per_clf = Perceptron(random_state=42, max_iter=100, tol=-np.infty)
per_clf.fit(X, y)

In [ ]:
y_pred = per_clf.predict([[2, 0.5]])
y_pred

### Multi-layer perceptron with TF Estimator API

TensorFlow includes an estimator API that provides pre-baked implementations of various kinds of estimators, as well as supporting the creation of custom estimators. One of the built in estimators is `DNNClassifier`, which can be used to train a multi-layer perceptron.

The following example trains a multi-layer perceptron to classify digits in the MNIST dataset:

In [ ]:
import tensorflow as tf

(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()

X_train = X_train.astype(np.float32).reshape(-1, 28*28) / 255.0
X_test = X_test.astype(np.float32).reshape(-1, 28*28) / 255.0

y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)

X_valid, X_train = X_train[:5000], X_train[5000:]
y_valid, y_train = y_train[:5000], y_train[5000:]

We can train a multi-layer perceptron using TensorFlow's high-level *TF.Learn* API:

In [ ]:
feature_columns = [tf.feature_column.numeric_column("X", shape=[28 * 28])]
feature_columns

In [ ]:
dnn_clf = tf.estimator.DNNClassifier(
    hidden_units=[300, 100], 
    n_classes=10, 
    feature_columns=feature_columns)

In [ ]:
input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"X": X_train},
    y=y_train,
    num_epochs=40,
    batch_size=50,
    shuffle=True)

dnn_clf.train(input_fn=input_fn)

In [ ]:
test_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"X": X_test},
    y=y_test,
    shuffle=False)

eval_results = dnn_clf.evaluate(input_fn=test_input_fn)
eval_results

### Using raw TensorFlow

TODO

In [ ]:
n_inputs = 28 * 28
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10

In [ ]:
X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int64, shape=(None), name="y")

In [ ]:
def neuron_layer(X, n_neurons, name, activation=None):
    with tf.name_scope(name):
        n_inputs = int(X.get_shape()[1])
        # Randomly initialize weights using a stddev based on number of inputs
        stddev = 2 / np.sqrt(n_inputs)
        init = tf.truncated_normal((n_inputs, n_neurons), stddev=stddev)
        W = tf.Variable(init, name="weights")
        b = tf.Variable(tf.zeros([n_neurons]), name="biases")
        # Create a subgraph to compute z = X * W + b
        z = tf.matmul(X, W) + b
        # Optional activation function
        if activation == 'relu':
            return tf.nn.relu(z)
        else:
            return z

In [ ]:
with tf.name_scope("dnn"):
    hidden1 = neuron_layer(X, n_hidden1, "hidden1", activation="relu")
    hidden2 = neuron_layer(hidden1, n_hidden2, "hidden2", activation="relu")
    logits = neuron_layer(hidden2, n_outputs, "outputs")

Alternatively, we can use TensorFlow's built-in `fully_connected` function to define dense layers:

In [ ]:
from tensorflow.contrib.layers import fully_connected

with tf.name_scope('dnn'):
    hidden1 = fully_connected(X, n_hidden1, scope="hidden1")
    hidden2 = fully_connected(hidden1, n_hidden2, scope="hidden2")
    logits = fully_connected(hidden2, n_outputs, scope="outputs", activation_fn=None)

Define the cost function:

In [ ]:
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")

In [ ]:
learning_rate = 0.01

with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

In [ ]:
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

In [ ]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [ ]:
def shuffle_batch(X, y, batch_size):
    rnd_idx = np.random.permutation(len(X))
    n_batches = len(X) // batch_size
    for batch_idx in np.array_split(rnd_idx, n_batches):
        X_batch, y_batch = X[batch_idx], y[batch_idx]
        yield X_batch, y_batch

In [ ]:
n_epochs = 400
batch_size = 50

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        acc_batch = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_val = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
        print(epoch, "Batch accuracy:", acc_batch, "Val accuracy:", acc_val)

    save_path = saver.save(sess, "./my_model_final.ckpt")